In [1]:
import torch
from ettcl.core.pipelines import ClassificationPipeline
from datasets import load_from_disk, Split
from sklearn.metrics import accuracy_score

text_column = 'text'
label_column = 'label'

In [2]:
dataset = load_from_disk("/home/IAIS/hiser/data/trec-6")
dataset.set_format('torch')
train_dataset = dataset[Split.TRAIN]
test_dataset = dataset[Split.TEST]

label_names = train_dataset.features[label_column].names

In [3]:
pipeline = ClassificationPipeline(
    "bert-base-uncased",
    architecture='colbert',
    indexer_kwargs={'num_partitions': 2048},
    searcher_kwargs={'ncells': 16},
    k=5,
)
pipeline.train_index(train_dataset, index_path="outputs/index_colbert")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing ColBERTModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing ColBERTModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ColBERTModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
preds = pipeline.predict(test_dataset[text_column], test_dataset[label_column])
accuracy_score(test_dataset[label_column], preds)

Setting TOKENIZERS_PARALLELISM=false for forked processes.


Searching (num_proc=2):   0%|          | 0/500 [00:00<?, ? examples/s]

0.902

In [5]:
false_preds = torch.where(preds != test_dataset[label_column])[0]
false_preds

tensor([ 11,  16,  27,  42,  49,  54,  75,  94,  99, 106, 118, 127, 133, 135,
        140, 158, 167, 170, 175, 188, 190, 191, 201, 223, 227, 230, 236, 240,
        249, 251, 255, 280, 291, 325, 361, 363, 376, 384, 389, 398, 405, 414,
        422, 425, 460, 467, 488, 489, 496])

In [6]:
pipeline.explain_prediction(11, 0)

Query 11: What person 's head is on a dime ?
True Label: HUM, predicted label: NUM
Match 0: How many grooves are on a dime ?
